# My Server の Container イメージをバックアップする
<HR>

現在実行中の「My Server」 の Container イメージを取得して、ファイルに保存します。

# Container イメージのバックアップ

* バックアップファイルは、ディレクトリ `$HOME/images` に保存されます。
* バックアップファイル名は、`server-container-YYYYMMDD-HHMMSS.tar` 形式で生成されます。

In [ ]:
import os
target_hub = ['-i', os.path.expanduser('~/ansible/inventory'), 'ch-master']

!ansible -m ping {' '.join(target_hub)}

**上記セルが動作しない場合**

```
The authenticity of host 'xxx.xxx.205.128 (xxx.xxx.205.128)' can't be established.
ECDSA key fingerprint is SHA256:qjPDx7y/926gHJL9+SgMGKpicRORzffk1/xiUyIP00w.
Are you sure you want to continue connecting (yes/no)?
```
（IPアドレスと、fingerprintは例です）

となり実行中のまま状態変化しなくなる場合は、JupyterのTerminalから、

```
$ ssh xxx.xxx.205.128
```

を実行し、ECDSA key fingerprintが `SHA256:qjPDx7y/926gHJL9+SgMGKpicRORzffk1/xiUyIP00w` であることを確認してyesを実行し、上記のセルを停止の上再実行してください。

# 初期ユーザー名の設定

JupyterHubのユーザー用のイメージには、初期ユーザーが存在している必要があります。
初期ユーザーのユーザー名やユーザーIDは、コンテナ起動後に、実際のユーザーのものに変更されるため、実行中のコンテナ上には存在しません。

このnotebookでは、イメージ保存前に初期ユーザーを復元する処理を行います。

https://github.com/jupyter/docker-stacks をベースとしたイメージの場合は`jovyan`ユーザーが、https://github.com/NII-cloud-operation/Jupyter-LC_docker をベースとしたイメージの場合は、`bit_kun`ユーザーを指定します。

以下のセルでは、ホームディレクトリの存在から初期ユーザーを判別して設定します。

In [ ]:
import os.path

if os.path.exists('/home/jovyan'):
    init_user='jovyan'
elif os.path.exists('/home/bit_kun'):
    init_user='bit_kun'
init_user

# バックアップの実行

このMy Serverを実行中のコンテナを特定します。

In [ ]:
def get_container(name):
    import subprocess
    try:
        sid = subprocess.check_output(['ansible', '-b', '-a', 'docker service ps {} -q'.format(name)] + target_hub)
        sid = sid.decode('utf-8').split('\n')[1].strip()
        cinfo = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker inspect --format "{% raw %} {{.NodeID}} {{.Status.ContainerStatus.ContainerID}} {% endraw %}" ' + sid
            ] + target_hub)
        nodeid, cid = cinfo.decode('utf-8').split('\n')[1].strip().split()
        nodeip = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker node inspect --format "{% raw %} {{.Status.Addr}} {% endraw %}" ' + nodeid
            ] + target_hub)
        nodeip = nodeip.decode('utf-8').split('\n')[1].split()[0]
        return (nodeip, cid)
    except subprocess.CalledProcessError as e:
        print(e.output.decode('utf-8'))
        raise

container = get_container('jupyter-' + os.environ['JUPYTERHUB_USER'])
container

In [ ]:
target_container = ['-i', os.path.expanduser('~/ansible/inventory'), container[0]]
target_container

コンテナのイメージを保存します。

In [ ]:
import subprocess
from datetime import datetime
import getpass
import re

now = datetime.now()
image_name = now.strftime('server-container-%Y%m%d-%H%M%S')
filename = image_name + '.tar'

!mkdir -p ~/images

print('Committing the container to image')
image_id = subprocess.check_output(['ansible', '-b', '-a', 
                                    'docker commit {}'.format(container[1])]
                                   + target_container)

base_image_id = image_id.decode('utf-8').split('\n')[1].strip()
print('Committed Image: {}'.format(base_image_id))

dockerfile = '''FROM {base}
USER root
WORKDIR /
RUN useradd -N -s /bin/bash {init_user} && \
    userdel {current_user} && rm -rf /home/{current_user}
USER {init_user}
'''.format(base=base_image_id, init_user=init_user, current_user=getpass.getuser()).replace('\n', '\\n')

print('Building image: {}'.format(image_name))
build_result = subprocess.check_output(['ansible', '-b', '-m', 'shell', '-a',
                                        'echo -e "{}" | docker build -t {} -'.format(dockerfile, image_name)]
                                       + target_container)
print(build_result.decode('utf-8'))
image_id = None
for l in build_result.decode('utf-8').split('\n'):
    m = re.match(r'^Successfully built ([0-9a-f]+)$', l)
    if m:
        image_id = m.group(1)

assert image_id is not None

print('Built image: {} ({})'.format(image_name, image_id))

print('Saving the image to the file: ~/images/{}'.format(filename))
subprocess.check_output(['ansible', '-b', '-a', 
                         'docker save -o /jupyter/users/{}/images/{} {}'.format(
                             os.environ['JUPYTERHUB_USER'], filename, image_name
                         )] + target_container)
print('Completed')